### Linear and Multilinear Functions

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [2]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(0.929 𝐞₁ + 0.148 𝐞₂ + 0.211 𝐞₃ + 1.45 𝐞₄,
 0.929 𝐞₁ + 0.148 𝐞₂ + 0.211 𝐞₃ + 1.45 𝐞₄)

In [3]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(-5.03 𝐞₁ + 1.38 𝐞₂ + 1.43 𝐞₃ + -0.483 𝐞₄,
 -5.03 𝐞₁ + 1.38 𝐞₂ + 1.43 𝐞₃ + -0.483 𝐞₄)

In [4]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [5]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(-1.64 𝐞₁ + 1.33 𝐞₂ + 1.92 𝐞₃ + 0.524 𝐞₄,
 -1.64 𝐞₁ + 1.33 𝐞₂ + 1.92 𝐞₃ + 0.524 𝐞₄)

In [6]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.0862 + -0.795 𝐞₁ + -1.23 𝐞₂ + 3.6 𝐞₃ + 1.7 𝐞₄ + 0.929 𝐞₁₂ + -1.49 𝐞₁₃ + -0.852 𝐞₁₄ + 1.91 𝐞₂₃ + 0.667 𝐞₂₄ + 0.681 𝐞₃₄,
 0.0862 + -0.795 𝐞₁ + -1.23 𝐞₂ + 3.6 𝐞₃ + 1.7 𝐞₄ + 0.929 𝐞₁₂ + -1.49 𝐞₁₃ + -0.852 𝐞₁₄ + 1.91 𝐞₂₃ + 0.667 𝐞₂₄ + 0.681 𝐞₃₄)

In [7]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(-4.41 𝐞₁₂₃ + -4.19 𝐞₁₂₄ + 2.83 𝐞₁₃₄ + -5.88 𝐞₂₃₄,
 -4.41 𝐞₁₂₃ + -4.19 𝐞₁₂₄ + 2.83 𝐞₁₃₄ + -5.88 𝐞₂₃₄)

In [8]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.0665 + -1.55 𝐞₁ + 0.817 𝐞₂ + 2.31 𝐞₃ + 0.723 𝐞₄ + 6.75 𝐞₁₂ + -7.03 𝐞₁₃ + -2.64 𝐞₁₄ + 6.72 𝐞₂₃ + -2.29 𝐞₂₄ + 4.66 𝐞₃₄ + 21.4 𝐞₁₂₃ + 21.4 𝐞₁₂₄ + -13.0 𝐞₁₃₄ + 28.0 𝐞₂₃₄ + -2.63 𝐞₁₂₃₄,
 0.0665 + -1.55 𝐞₁ + 0.817 𝐞₂ + 2.31 𝐞₃ + 0.723 𝐞₄ + 6.75 𝐞₁₂ + -7.03 𝐞₁₃ + -2.64 𝐞₁₄ + 6.72 𝐞₂₃ + -2.29 𝐞₂₄ + 4.66 𝐞₃₄ + 21.4 𝐞₁₂₃ + 21.4 𝐞₁₂₄ + -13.0 𝐞₁₃₄ + 28.0 𝐞₂₃₄ + -2.63 𝐞₁₂₃₄)

In [9]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.0862 + 1.43 𝐞₁ + -0.834 𝐞₂ + 1.87 𝐞₃ + 3.19 𝐞₄ + -1.0 𝐞₁₂ + 0.775 𝐞₁₃ + 0.264 𝐞₁₄ + -0.21 𝐞₂₃ + 1.1 𝐞₂₄ + -0.767 𝐞₃₄ + -12.4 𝐞₁₂₃ + -10.5 𝐞₁₂₄ + 5.41 𝐞₁₃₄ + -14.2 𝐞₂₃₄ + -0.854 𝐞₁₂₃₄,
 0.0862 + 1.43 𝐞₁ + -0.834 𝐞₂ + 1.87 𝐞₃ + 3.19 𝐞₄ + -1.0 𝐞₁₂ + 0.775 𝐞₁₃ + 0.264 𝐞₁₄ + -0.21 𝐞₂₃ + 1.1 𝐞₂₄ + -0.767 𝐞₃₄ + -12.4 𝐞₁₂₃ + -10.5 𝐞₁₂₄ + 5.41 𝐞₁₃₄ + -14.2 𝐞₂₃₄ + -0.854 𝐞₁₂₃₄)

In [10]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.0862 + 1.43 𝐞₁ + -0.834 𝐞₂ + 1.0 𝐞₁₂,
 0.0862 + 1.43 𝐞₁ + -0.834 𝐞₂ + 1.0 𝐞₁₂)

In [11]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [12]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.0862 + 0.341 𝐞₁ + -1.82 𝐞₂ + 1.46 𝐞₃ + -0.194 𝐞₄ + 0.929 𝐞₁₂ + -0.962 𝐞₁₃ + 0.333 𝐞₁₄ + 1.15 𝐞₂₃ + -1.24 𝐞₂₄ + 0.875 𝐞₃₄,
 0.0862 + 0.341 𝐞₁ + -1.82 𝐞₂ + 1.46 𝐞₃ + -0.194 𝐞₄ + 0.929 𝐞₁₂ + -0.962 𝐞₁₃ + 0.333 𝐞₁₄ + 1.15 𝐞₂₃ + -1.24 𝐞₂₄ + 0.875 𝐞₃₄)

In [13]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.0506 + 1.34 𝐞₁ + 0.73 𝐞₂ + 0.0458 𝐞₃ + 0.0784 𝐞₄ + 3.77 𝐞₁₂ + 4.48 𝐞₁₃ + -5.86 𝐞₁₄ + 3.63 𝐞₂₃ + -4.55 𝐞₂₄ + 0.184 𝐞₃₄ + 55.2 𝐞₁₂₃ + -45.9 𝐞₁₂₄ + 27.2 𝐞₁₃₄ + 18.8 𝐞₂₃₄ + -7.53 𝐞₁₂₃₄,
 0.0506 + 1.34 𝐞₁ + 0.73 𝐞₂ + 0.0458 𝐞₃ + 0.0784 𝐞₄ + 3.77 𝐞₁₂ + 4.48 𝐞₁₃ + -5.86 𝐞₁₄ + 3.63 𝐞₂₃ + -4.55 𝐞₂₄ + 0.184 𝐞₃₄ + 55.2 𝐞₁₂₃ + -45.9 𝐞₁₂₄ + 27.2 𝐞₁₃₄ + 18.8 𝐞₂₃₄ + -7.53 𝐞₁₂₃₄)

In [14]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(-3.51 𝐞₁ + -1.76 𝐞₂ + 1.33 𝐞₃ + -1.37 𝐞₄,
 0.496 𝐞₁ + -0.566 𝐞₂ + -0.625 𝐞₃ + -0.558 𝐞₄,
 -3.51 𝐞₁ + -1.76 𝐞₂ + 1.33 𝐞₃ + -1.37 𝐞₄)

In [15]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(-8.23 𝐞₁ + 2.66 𝐞₂ + -8.11 𝐞₃ + -15.1 𝐞₄,
 1.03 𝐞₁ + -8.61 𝐞₂ + -13.2 𝐞₃ + -10.2 𝐞₄,
 -8.23 𝐞₁ + 2.66 𝐞₂ + -8.11 𝐞₃ + -15.1 𝐞₄)

In [16]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

3.15 3.15
-0.608 -0.608
-12.8 -12.8


In [17]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(-3.51 𝐞₁ + -1.76 𝐞₂ + 1.33 𝐞₃ + -1.37 𝐞₄,
 -3.51 𝐞₁ + -1.76 𝐞₂ + 1.33 𝐞₃ + -1.37 𝐞₄)

In [18]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, -4.26 𝐞₁₂₃₄, -4.26 𝐞₁₂₃₄)

In [19]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(-4.26, -4.26, -4.26)

In [20]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-2.29 𝐞₁₃, -2.29 𝐞₁₃, -2.29 𝐞₁₃)

In [21]:
#1.17a
outermorphism(g, I, alg, h), Ip

(2.29 𝐞₂₃, 1 𝐞₂₃)

In [22]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(2.29 𝐞₁₂, 1 𝐞₁₂)

In [23]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

2.2915279009086413

In [24]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.183 + 0.698 𝐞₁ + 0.922 𝐞₂ + 0.406 𝐞₁₂,
 0.183 + 0.698 𝐞₁ + 0.922 𝐞₂ + 0.521 𝐞₃ + 0.203 𝐞₄ + 0.406 𝐞₁₂ + 0.922 𝐞₁₃ + 0.163 𝐞₁₄ + 0.646 𝐞₂₃ + 0.208 𝐞₂₄ + 0.0817 𝐞₃₄ + 0.385 𝐞₁₂₃ + 0.305 𝐞₁₂₄ + 0.707 𝐞₁₃₄ + 0.637 𝐞₂₃₄ + 0.286 𝐞₁₂₃₄)

In [25]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.0617 + 0.37 𝐞₂ + 0.598 𝐞₃ + 0.245 𝐞₂₃,
 0.0617 + 0.639 𝐞₁ + 0.37 𝐞₂ + 0.598 𝐞₃ + 0.664 𝐞₄ + 0.911 𝐞₁₂ + 0.301 𝐞₁₃ + 0.958 𝐞₁₄ + 0.245 𝐞₂₃ + 0.326 𝐞₂₄ + 0.127 𝐞₃₄ + 0.414 𝐞₁₂₃ + 0.992 𝐞₁₂₄ + 0.285 𝐞₁₃₄ + 0.0436 𝐞₂₃₄ + 0.231 𝐞₁₂₃₄)

In [26]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(-1.25 𝐞₁₂₃ + 0.673 𝐞₁₂₄ + -0.179 𝐞₁₃₄ + 0.038 𝐞₂₃₄,
 -1.25 𝐞₁₂₃ + 0.673 𝐞₁₂₄ + -0.179 𝐞₁₃₄ + 0.038 𝐞₂₃₄,
 16.2 + 3.8 𝐞₁₂ + 11.8 𝐞₁₃ + -4.73 𝐞₁₄ + -5.44 𝐞₂₃ + -0.718 𝐞₂₄ + 8.26 𝐞₃₄,
 16.2 + 3.8 𝐞₁₂ + 11.8 𝐞₁₃ + -4.73 𝐞₁₄ + -5.44 𝐞₂₃ + -0.718 𝐞₂₄ + 8.26 𝐞₃₄)

In [27]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-4.67 + 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄,
 -4.67 + -4.72 𝐞₁ + 0.247 𝐞₂ + -0.641 𝐞₃ + 0.272 𝐞₄ + 1.62 𝐞₁₂ + 1.09 𝐞₁₃ + 3.61 𝐞₁₄ + 1.25 𝐞₂₃ + 1.77 𝐞₂₄ + 1.21 𝐞₃₄ + 0.707 𝐞₁₂₃ + 1.48 𝐞₁₂₄ + -1.33 𝐞₁₃₄ + 1.08 𝐞₂₃₄ + -1.02 𝐞₁₂₃₄)

In [28]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([0.543 𝐞₁ + 2.28 𝐞₂ + 2.04 𝐞₃ + -2.11 𝐞₄,
  0.886 𝐞₁ + 2.24 𝐞₂ + 0.101 𝐞₃ + -1.62 𝐞₄,
  0.825 𝐞₁ + 2.55 𝐞₂ + 1.58 𝐞₃ + -2.18 𝐞₄,
  1.25 𝐞₁ + 1.45 𝐞₂ + 1.09 𝐞₃ + -0.696 𝐞₄],
 [0.543 𝐞₁ + 2.28 𝐞₂ + 2.04 𝐞₃ + -2.11 𝐞₄,
  0.886 𝐞₁ + 2.24 𝐞₂ + 0.101 𝐞₃ + -1.62 𝐞₄,
  0.825 𝐞₁ + 2.55 𝐞₂ + 1.58 𝐞₃ + -2.18 𝐞₄,
  1.25 𝐞₁ + 1.45 𝐞₂ + 1.09 𝐞₃ + -0.696 𝐞₄])

In [29]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-4.67, -4.67, -4.67)

In [30]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(2.53 + -5.84 𝐞₁₂ + -2.76 𝐞₁₃ + 0.34 𝐞₁₄ + -4.11 𝐞₂₃ + 1.94 𝐞₂₄ + 1.24 𝐞₃₄ + -3.14 𝐞₁₂₃₄,
 2.53 + -5.84 𝐞₁₂ + -2.76 𝐞₁₃ + 0.34 𝐞₁₄ + -4.11 𝐞₂₃ + 1.94 𝐞₂₄ + 1.24 𝐞₃₄ + -3.14 𝐞₁₂₃₄)

In [31]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(-20.90244475689767, -20.902444756897662, -20.90244475689768)

In [32]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(23.9 + 8.83 𝐞₁₂ + 37.0 𝐞₁₃ + -15.3 𝐞₁₄ + -9.34 𝐞₂₃ + -1.74 𝐞₂₄ + 19.6 𝐞₃₄,
 23.9 + 8.83 𝐞₁₂ + 37.0 𝐞₁₃ + -15.3 𝐞₁₄ + -9.34 𝐞₂₃ + -1.74 𝐞₂₄ + 19.6 𝐞₃₄)

In [33]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(2.4e+02, 239.59130512023478)

In [34]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(-5.39 𝐞₁ + 15.9 𝐞₂ + 17.8 𝐞₃ + -14.1 𝐞₄,
 -5.39 𝐞₁ + 15.9 𝐞₂ + 17.8 𝐞₃ + -14.1 𝐞₄,
 0.715 𝐞₁ + 0.343 𝐞₂ + 0.597 𝐞₃ + 0.542 𝐞₄,
 0.715 𝐞₁ + 0.343 𝐞₂ + 0.597 𝐞₃ + 0.542 𝐞₄)

In [35]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

-2.13e-14 𝐞₁ + -1.07e-14 𝐞₂ + -1.42e-14 𝐞₃

In [36]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([ 1.86309835+0.j        , -1.91437555+0.j        ,
        -2.30709094+0.73336525j, -2.30709094-0.73336525j]),
 array([ 1.86309835+0.j        , -2.30709094+0.73336525j,
        -2.30709094-0.73336525j, -1.91437555+0.j        ]))

In [37]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.18838056, 0.18838056, 0.18838056, 0.18838056, 0.18838056,
       0.18838056, 0.18838056, 0.18838056])

In [38]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(0.564 𝐞₁ + 2.46 𝐞₂ + 1.53 𝐞₃ + -1.59 𝐞₄,
 0.564 𝐞₁ + 2.46 𝐞₂ + 1.53 𝐞₃ + -1.59 𝐞₄)

In [39]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(2.5304256177177464, 2.5304256177177464)

In [40]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(16.156754003443865, (16.156754003443872-4.440892098500626e-16j))

In [41]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(-17.663701094334066, (-17.663701094334037+0j))

In [42]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(-20.9 𝐞₁₂₃₄, -20.9 𝐞₁₂₃₄, -20.9 𝐞₁₂₃₄)

In [43]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(28.9 𝐞₁₂₃₄, 28.9 𝐞₁₂₃₄)

In [44]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(7.64 𝐞₁₂, 7.64 𝐞₁₂)

In [45]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-23.278817018794324, -23.278817018794324)

In [46]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-0.451 𝐞₁ + 1.2 𝐞₂, -0.451 𝐞₁ + -0.0324 𝐞₂)

In [47]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(0.288 𝐞₃, 0.288 𝐞₃)

In [48]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(-0.288 𝐞₁₂, -0.288 𝐞₁₂)

In [49]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([0, 1], [(2, 3)]),
 array([ 1.86309835+0.j        , -1.91437555+0.j        ,
        -2.30709094+0.73336525j, -2.30709094-0.73336525j]))

In [50]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[1.86309835 1.86309835 1.86309835 1.86309835] [ 3.7283291   1.2659413   0.34985881 22.80467403]
[-1.91437555 -1.91437555 -1.91437555 -1.91437555] [ -3.35039455   0.15904412  -1.53446921 -21.26335661]
[5.8604932 5.8604932 5.8604932 5.8604932 5.8604932 5.8604932] [ 16.62103831 -30.08810242   7.94133547   7.93339231 -11.02777908
   8.83283833]


In [51]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[ 0.83794106  3.61009701  5.43724279 -0.60745579] [1.86309835 1.86309835 1.86309835 1.86309835]
[ 0.24975789 -2.00973419 -2.88449771 13.75309682] [-1.91437555 -1.91437555 -1.91437555 -1.91437555]
[ -8.91904834  -7.45759735  -7.25941105 -98.60866587   3.0440755
  10.73042757] [5.8604932 5.8604932 5.8604932 5.8604932 5.8604932 5.8604932]


In [52]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, -7.04e-16, 1.0]

In [53]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(6.49 𝐞₁₂₃, 6.49 𝐞₁₂₃), (-2.68 𝐞₄, -2.68 𝐞₄)]

In [54]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[1.81423615 1.81423615 1.81423615] [3.48734895 1.1678115  0.59948192]
[-1.70260113 -1.70260113 -1.70260113] [-3.14129259  0.81384943 -1.82125542]
[-2.10062082 -2.10062082 -2.10062082] [-2.96504947 38.14231471 -1.97864426]
[] []
[] []
[] []
[] []
[-2.67647328] [-2.67647328]


In [55]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[ 0.1641324 ,  1.38808496,  1.09692552, -0.30673852],
        [ 1.38808496, -0.5446416 ,  1.10219199,  1.21874256],
        [ 1.09692552,  1.10219199, -1.6084766 ,  0.54624952],
        [-0.30673852,  1.21874256,  0.54624952, -2.67647328]]),
 1.08 𝐞₁ + 2.12 𝐞₂ + 0.498 𝐞₃ + -0.925 𝐞₄,
 1.08 𝐞₁ + 2.12 𝐞₂ + 0.498 𝐞₃ + -0.925 𝐞₄)

In [56]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄,
 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄)

In [57]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[ 1.11022302e-16, -6.17170096e-01, -1.64785884e-01,
         -3.84426889e-01],
        [ 6.17170096e-01,  5.55111512e-17, -3.01212808e-01,
          1.42821557e-01],
        [ 1.64785884e-01,  3.01212808e-01,  1.11022302e-16,
          1.49666632e+00],
        [ 3.84426889e-01, -1.42821557e-01, -1.49666632e+00,
         -2.22044605e-16]]),
 -0.518 𝐞₁ + 0.339 𝐞₂ + 1.03 𝐞₃ + -0.667 𝐞₄,
 -0.518 𝐞₁ + 0.339 𝐞₂ + 1.03 𝐞₃ + -0.667 𝐞₄)

In [58]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(1.12,
 1.12,
 2.33e-10 𝐞₁₂ + 2.91e-10 𝐞₁₄ + -4.37e-10 𝐞₂₃ + -2.33e-10 𝐞₂₄ + -1.46e-10 𝐞₃₄)

In [59]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.08 𝐞₁ + 2.12 𝐞₂ + 0.498 𝐞₃ + -0.925 𝐞₄,
 1.08 𝐞₁ + 2.12 𝐞₂ + 0.498 𝐞₃ + -0.925 𝐞₄)

In [60]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-1.18e+02 𝐞₁ + -7.27e-15 𝐞₂ + -1e-14 𝐞₃ + -6.17e-15 𝐞₄,
 1.82e-15 𝐞₁ + -29.5 𝐞₂ + -2.24e-15 𝐞₃ + 2.58e-15 𝐞₄,
 4.93e-16 𝐞₁ + 4.4e-16 𝐞₂ + -5.8 𝐞₃ + 1.62e-15 𝐞₄,
 8.33e-17 𝐞₁ + -1.39e-16 𝐞₂ + -4.44e-16 𝐞₃ + -1.59 𝐞₄]

In [61]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [62]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -4.67 + -2.22e-16 𝐞₁₃ + 1.67e-16 𝐞₁₄ + -4.44e-16 𝐞₂₃ + -2.22e-16 𝐞₂₄ + 3.33e-16 𝐞₃₄,
 -0.377 + -8.88e-16 𝐞₁₂ + 8.88e-16 𝐞₁₃ + -5.55e-17 𝐞₁₄ + 1.78e-15 𝐞₂₃ + -2.22e-16 𝐞₃₄ + -2.22e-16 𝐞₁₂₃₄,
 19.8 + 3.55e-15 𝐞₁₂ + -1.78e-15 𝐞₁₃ + 8.88e-16 𝐞₁₄ + -4.44e-16 𝐞₂₃ + 1.33e-15 𝐞₂₄ + -6.66e-16 𝐞₃₄,
 -14.6]

In [63]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([ 2.03463272, -3.4894681 , -2.32968613, -0.88093757]),
 array([-3.4894681 , -2.32968613, -0.88093757,  2.03463272]))

In [64]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.28 𝐞₁ + -0.422 𝐞₂ + -0.162 𝐞₃ + 0.847 𝐞₄,
  0.148 𝐞₁ + 0.435 𝐞₂ + -0.888 𝐞₃ + -0.0025 𝐞₄,
  -0.678 𝐞₁ + 0.514 𝐞₂ + 0.137 𝐞₃ + 0.507 𝐞₄,
  -0.663 𝐞₁ + -0.607 𝐞₂ + -0.407 𝐞₃ + -0.161 𝐞₄],
 [0.28 𝐞₁ + -0.422 𝐞₂ + -0.162 𝐞₃ + 0.847 𝐞₄,
  -0.148 𝐞₁ + -0.435 𝐞₂ + 0.888 𝐞₃ + 0.0025 𝐞₄,
  0.678 𝐞₁ + -0.514 𝐞₂ + -0.137 𝐞₃ + -0.507 𝐞₄,
  0.663 𝐞₁ + 0.607 𝐞₂ + 0.407 𝐞₃ + 0.161 𝐞₄])

In [65]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-3.4894681 -3.4894681 -3.4894681 -3.4894681]
[12.17638763 12.17638763 12.17638763 12.17638763 12.17638763 12.17638763]


In [66]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([-0.678 𝐞₁ + 0.514 𝐞₂ + 0.137 𝐞₃ + 0.507 𝐞₄,
  -0.663 𝐞₁ + -0.607 𝐞₂ + -0.407 𝐞₃ + -0.161 𝐞₄,
  0.184 𝐞₁₂ + -0.225 𝐞₁₃ + -0.126 𝐞₁₄ + 0.445 𝐞₂₃ + -0.367 𝐞₂₄ + 0.753 𝐞₃₄],
 [-0.184 𝐞₁₂ + 0.225 𝐞₁₃ + 0.126 𝐞₁₄ + -0.445 𝐞₂₃ + 0.367 𝐞₂₄ + -0.753 𝐞₃₄,
  0.678 𝐞₁ + -0.514 𝐞₂ + -0.137 𝐞₃ + -0.507 𝐞₄,
  0.663 𝐞₁ + 0.607 𝐞₂ + 0.407 𝐞₃ + 0.161 𝐞₄])

In [67]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.97501058, -0.15161579,  0.15943817, -0.03076524],
        [-0.15161579, -0.08011699, -0.96734296,  0.18665881],
        [ 0.15943817, -0.96734296,  0.01725154, -0.19628919],
        [-0.03076524,  0.18665881, -0.19628919, -0.96212398]]),
 [1.0000000000000007, 0.0, 0.0, 1.0000000000000007],
 array([-1.,  1., -1., -1.]))

In [68]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(-0.671 𝐞₁ + -0.612 𝐞₂ + -0.314 𝐞₃ + -0.596 𝐞₄,
 -0.671 𝐞₁ + -0.612 𝐞₂ + -0.314 𝐞₃ + -0.596 𝐞₄)

In [69]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([-0.579 𝐞₁ + -0.657 𝐞₂ + -0.483 𝐞₃,
  -0.54 𝐞₁ + 0.437 𝐞₂ + 0.72 𝐞₃,
  -0.388 𝐞₁ + 0.0257 𝐞₂ + 0.921 𝐞₃,
  1.0 𝐞₄,
  1 𝐞₁₂₃],
 [0.2597375883159782, 1.2684960021774447, 1.21879647181827, 0.0, 0.0])

In [70]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(0.746, 0.746)

In [71]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(-0.518 𝐞₁ + 0.339 𝐞₂ + 1.03 𝐞₃ + -0.667 𝐞₄,
 -0.518 𝐞₁ + 0.339 𝐞₂ + 1.03 𝐞₃ + -0.667 𝐞₄,
 2.22e-16)

In [72]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(-1.16e-10 + 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄,
 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄,
 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄,
 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄)

In [73]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [74]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
  -0.784 𝐞₁₂₃₄],
 [5.55e-17 + 1.23 𝐞₁₂ + 0.33 𝐞₁₃ + 0.769 𝐞₁₄ + 0.602 𝐞₂₃ + -0.286 𝐞₂₄ + -2.99 𝐞₃₄,
  2.91 + -6.94e-18 𝐞₁₂ + 1.11e-16 𝐞₁₃ + 5.27e-16 𝐞₁₄ + 1.67e-16 𝐞₂₃ + -3.89e-16 𝐞₂₄ + -6.11e-16 𝐞₃₄ + -3.14 𝐞₁₂₃₄,
  6.38e-16 + 2.35 𝐞₁₂ + -0.224 𝐞₁₃ + -0.473 𝐞₁₄ + -0.603 𝐞₂₃ + 0.259 𝐞₂₄ + -0.968 𝐞₃₄,
  0.615])

In [75]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
 -1.17 𝐞₁₂ + 0.112 𝐞₁₃ + 0.236 𝐞₁₄ + 0.302 𝐞₂₃ + -0.129 𝐞₂₄ + 0.484 𝐞₃₄]

In [76]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [77]:
x|F, fm(x), x|F|F, fm(fm(x))

(-0.518 𝐞₁ + 0.339 𝐞₂ + 1.03 𝐞₃ + -0.667 𝐞₄,
 -0.518 𝐞₁ + 0.339 𝐞₂ + 1.03 𝐞₃ + -0.667 𝐞₄,
 -0.123 𝐞₁ + -0.726 𝐞₂ + -0.982 𝐞₃ + -1.79 𝐞₄,
 -0.123 𝐞₁ + -0.726 𝐞₂ + -0.982 𝐞₃ + -1.79 𝐞₄)

In [78]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 2.906974574808199, 0.6152338129913806],
 [1,
  -5.551115123125783e-17,
  2.906974574808199,
  -6.38378239159465e-16,
  0.6152338129913808],
 array([-2.67716677, -0.2298078 ]),
 array([1.63620499, 0.47938273]),
 array([-1.00060031e-16+1.63620499j, -1.00060031e-16-1.63620499j,
         1.27815607e-16+0.47938273j,  1.27815607e-16-0.47938273j]),
 [1,
  5.813949149616398,
  9.680968804564072,
  3.576938103856493,
  0.3785126446479134])

In [79]:
Ck[1], Ck[1] | Ck[2]

(0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
 -1.17 𝐞₁₂ + 0.112 𝐞₁₃ + 0.236 𝐞₁₄ + 0.302 𝐞₂₃ + -0.129 𝐞₂₄ + 0.484 𝐞₃₄)

In [80]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
 0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
 -1.39e-17 + 6.94e-18 𝐞₁₂ + 4.16e-17 𝐞₁₃ + -8.67e-18 𝐞₁₄ + 5.2e-18 𝐞₂₃ + -2.78e-17 𝐞₂₄ + 1.39e-17 𝐞₃₄ + -0.784 𝐞₁₂₃₄,
 -1.39e-17 + -6.94e-18 𝐞₁₂ + -5.55e-17 𝐞₁₃ + 1.39e-17 𝐞₁₄ + 2.78e-17 𝐞₂₄ + -1.39e-17 𝐞₃₄ + -0.784 𝐞₁₂₃₄,
 -2.91 + 2.78e-17 𝐞₁₄ + -1.57 𝐞₁₂₃₄,
 -0.23 + 8.67e-19 𝐞₁₃ + -1.3e-18 𝐞₁₄ + -1.73e-17 𝐞₁₂₃₄,
 -2.68 + 1.67e-16 𝐞₁₂₃₄)

In [81]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-2.67716677225867, -0.22980780254952915], array([-2.67716677, -0.2298078 ]))

In [82]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(0.422 𝐞₁₂ + -0.0612 𝐞₁₃ + -0.133 𝐞₁₄ + -0.151 𝐞₂₃ + 0.0662 𝐞₂₄ + -0.0573 𝐞₃₄,
 0.422 𝐞₁₂ + -0.0612 𝐞₁₃ + -0.133 𝐞₁₄ + -0.151 𝐞₂₃ + 0.0662 𝐞₂₄ + -0.0573 𝐞₃₄,
 0.195 𝐞₁₂ + 0.226 𝐞₁₃ + 0.517 𝐞₁₄ + 0.453 𝐞₂₃ + -0.209 𝐞₂₄ + -1.44 𝐞₃₄,
 0.195 𝐞₁₂ + 0.226 𝐞₁₃ + 0.517 𝐞₁₄ + 0.453 𝐞₂₃ + -0.209 𝐞₂₄ + -1.44 𝐞₃₄)

In [83]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.linalg.solve(np.array([np.ones(m), Lk]), [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [84]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-0.505 + 2.1e-13 𝐞₁ + 1.37e-12 𝐞₂ + -7.3e-13 𝐞₃ + -5.47e-13 𝐞₄ + 2.06e-13 𝐞₁₂ + -5.07e-15 𝐞₁₃ + -6.23e-14 𝐞₁₄ + 6.76e-14 𝐞₂₃ + 1.48e-13 𝐞₂₄ + -1.52e-13 𝐞₃₄ + -3.25e-13 𝐞₁₂₃ + 3.07e-14 𝐞₁₂₄ + -1.36e-13 𝐞₁₃₄ + -3.34e-13 𝐞₂₃₄ + -8.72e-14 𝐞₁₂₃₄,
 [array([ 1.90848033e-13, -3.94853032e-14,  7.01406106e-14,  1.06038413e-13,
         -1.88792300e-13,  3.90365504e-14]),
  array([1., 1., 1., 1., 1., 1.]),
  array([8.63184448e-14, 8.63427724e-14, 8.62245341e-14, 8.62585006e-14,
         8.63363664e-14, 8.63088864e-14]),
  array([ 0.45228889, -2.18670658,  1.22930972,  0.81346465, -0.45730873,
          2.21097626])])

In [85]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

-8.88e-16 𝐞₁₂ + 8.88e-15 𝐞₁₃ + -1.78e-14 𝐞₁₄ + -2.84e-14 𝐞₂₃ + -1.42e-14 𝐞₂₄ + -4.26e-14 𝐞₃₄

In [86]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.199 𝐞₁ + -0.000495 𝐞₂ + -0.0715 𝐞₃ + 0.0314 𝐞₄,
 -0.199 𝐞₁ + -0.000495 𝐞₂ + -0.0715 𝐞₃ + 0.0314 𝐞₄ + 2.37e-18 𝐞₁₂₃ + 5.4e-18 𝐞₁₂₄ + 2.49e-19 𝐞₁₃₄ + 1.5e-17 𝐞₂₃₄)

In [87]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(0.422 𝐞₁₂ + -0.0612 𝐞₁₃ + -0.133 𝐞₁₄ + -0.151 𝐞₂₃ + 0.0662 𝐞₂₄ + -0.0573 𝐞₃₄,
 0.422 𝐞₁₂ + -0.0612 𝐞₁₃ + -0.133 𝐞₁₄ + -0.151 𝐞₂₃ + 0.0662 𝐞₂₄ + -0.0573 𝐞₃₄)

In [88]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [89]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [90]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(0.408 𝐞₁₂ + 0.0405 𝐞₁₃ + 0.719 𝐞₁₄ + 0.299 𝐞₂₃ + -0.212 𝐞₂₄ + -0.315 𝐞₃₄,
 0.408 𝐞₁₂ + 0.0405 𝐞₁₃ + 0.719 𝐞₁₄ + 0.299 𝐞₂₃ + -0.212 𝐞₂₄ + -0.315 𝐞₃₄)

In [91]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(0.408 𝐞₁₂ + 0.0405 𝐞₁₃ + 0.719 𝐞₁₄ + 0.299 𝐞₂₃ + -0.212 𝐞₂₄ + -0.315 𝐞₃₄,
 0.408 𝐞₁₂ + 0.0405 𝐞₁₃ + 0.719 𝐞₁₄ + 0.299 𝐞₂₃ + -0.212 𝐞₂₄ + -0.315 𝐞₃₄)

In [92]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(0.0969 𝐞₁₂ + -0.0141 𝐞₁₃ + -0.0305 𝐞₁₄ + -0.0348 𝐞₂₃ + 0.0152 𝐞₂₄ + -0.0132 𝐞₃₄,
 0.0969 𝐞₁₂ + -0.0141 𝐞₁₃ + -0.0305 𝐞₁₄ + -0.0348 𝐞₂₃ + 0.0152 𝐞₂₄ + -0.0132 𝐞₃₄)

In [93]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(1.72 𝐞₁ + -0.58 𝐞₂ + -2.22 𝐞₃ + 2.92 𝐞₄,
 -1.04 𝐞₁ + 0.385 𝐞₂ + -0.567 𝐞₃ + -0.633 𝐞₄,
 6.84 𝐞₁ + -0.473 𝐞₂ + 1.55 𝐞₃ + 10.4 𝐞₄,
 1.32 𝐞₁ + 1.46 𝐞₂ + 4.85 𝐞₃ + 3.24 𝐞₄)

In [94]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(6.99 𝐞₁ + -2.27 𝐞₂ + 1.32 𝐞₃ + 1.71 𝐞₄,
 6.99 𝐞₁ + -2.27 𝐞₂ + 1.32 𝐞₃ + 1.71 𝐞₄)

In [95]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(-1.89 𝐞₁ + -0.185 𝐞₂ + -0.894 𝐞₃ + 1.24 𝐞₄,
 -0.168 𝐞₁ + -0.352 𝐞₂ + -2.44 𝐞₃ + 2.47 𝐞₄ + -4.95e-17 𝐞₁₂₃ + -2.01e-17 𝐞₁₂₄ + 5.83e-18 𝐞₁₃₄ + -2.23e-16 𝐞₂₃₄)

In [96]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
 0.617 𝐞₁₂ + 0.165 𝐞₁₃ + 0.384 𝐞₁₄ + 0.301 𝐞₂₃ + -0.143 𝐞₂₄ + -1.5 𝐞₃₄,
 0.202 𝐞₁₂ + -0.0294 𝐞₁₃ + -0.0636 𝐞₁₄ + -0.0726 𝐞₂₃ + 0.0317 𝐞₂₄ + -0.0275 𝐞₃₄,
 0.202 𝐞₁₂ + -0.0294 𝐞₁₃ + -0.0636 𝐞₁₄ + -0.0726 𝐞₂₃ + 0.0317 𝐞₂₄ + -0.0275 𝐞₃₄)

In [97]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([-2.7703409 , -0.90528792, -2.51418724, -6.47320786, 22.23960277,
       -3.66735242])

In [98]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.00791421, 0.00791421, 0.00791421, 0.00791421, 0.00791421,
        0.00791421])]

In [99]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([-0.08896186524412794], array([0.00791421]), [0.007914213308357558])

In [100]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.102 𝐞₁ + -0.628 𝐞₂ + -0.608 𝐞₃ + -0.66 𝐞₄],
 [-0.102 𝐞₁ + -0.628 𝐞₂ + -0.608 𝐞₃ + -0.66 𝐞₄ + -6.94e-18 𝐞₁₂₃ + 3.47e-18 𝐞₁₂₄ + 2.78e-17 𝐞₁₃₄],
 [-0.102 𝐞₁ + -0.628 𝐞₂ + -0.608 𝐞₃ + -0.66 𝐞₄ + 1.39e-17 𝐞₁₂₃ + -1.73e-18 𝐞₁₂₄ + -2.78e-17 𝐞₁₃₄])

In [101]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.0773 𝐞₁ + 0.567 𝐞₂ + 0.747 𝐞₃ + 0.57 𝐞₄],
 [0.0773 𝐞₁ + 0.567 𝐞₂ + 0.747 𝐞₃ + 0.57 𝐞₄ + -1.39e-17 𝐞₁₂₃ + 1.73e-18 𝐞₁₂₄ + 2.78e-17 𝐞₁₃₄],
 [0.0773 𝐞₁ + 0.567 𝐞₂ + 0.747 𝐞₃ + 0.57 𝐞₄ + 6.94e-18 𝐞₁₂₃ + -3.47e-18 𝐞₁₂₄ + -2.78e-17 𝐞₁₃₄])

In [102]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.1000000000348897], [1.1000000000348895])

In [103]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([-0.089 + -0.0522 𝐞₁₂ + -0.163 𝐞₁₃ + -0.0396 𝐞₁₄ + -0.696 𝐞₂₃ + 0.0944 𝐞₂₄ + 0.823 𝐞₃₄],
 [-0.089 + -0.0522 𝐞₁₂ + -0.163 𝐞₁₃ + -0.0396 𝐞₁₄ + -0.696 𝐞₂₃ + 0.0944 𝐞₂₄ + 0.823 𝐞₃₄])

In [104]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.08896186524413081

In [105]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 4.37231338, -3.27231338])

In [106]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(-1.89 𝐞₁ + -0.185 𝐞₂ + -0.894 𝐞₃ + 1.24 𝐞₄,
 -1.89 𝐞₁ + -0.185 𝐞₂ + -0.894 𝐞₃ + 1.24 𝐞₄ + 6.72e-18 𝐞₁₂₃ + -6.83e-18 𝐞₁₂₄ + -2.73e-17 𝐞₁₃₄ + -5.38e-17 𝐞₂₃₄)

In [107]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-3.27231338, -0.08896186, -0.08896186,  4.37231338]),
 [-0.986 𝐞₁ + 0.0531 𝐞₂ + -0.0535 𝐞₃ + 0.151 𝐞₄,
  -0.0584 𝐞₁ + 0.0688 𝐞₂ + 0.994 𝐞₃ + -0.0536 𝐞₄,
  -0.15 𝐞₁ + -0.638 𝐞₂ + -0.00533 𝐞₃ + -0.755 𝐞₄,
  0.0512 𝐞₁ + -0.765 𝐞₂ + 0.0902 𝐞₃ + 0.636 𝐞₄])

In [108]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([10.70803486, 10.70803486, 10.70803486, 10.70803486]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([19.1171243, 19.1171243, 19.1171243, 19.1171243])]

In [109]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(-1.89 𝐞₁ + -0.185 𝐞₂ + -0.894 𝐞₃ + 1.24 𝐞₄,
 -1.89 𝐞₁ + -0.185 𝐞₂ + -0.894 𝐞₃ + 1.24 𝐞₄ + 1.37e-17 𝐞₁₂₃ + -1.59e-18 𝐞₁₂₄ + 3.17e-18 𝐞₁₃₄ + 5.77e-17 𝐞₂₃₄)

In [110]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.0862 + 0.419 𝐞₁ + 0.81 𝐞₂ + 0.859 𝐞₃ + 0.0122 𝐞₄ + 0.182 𝐞₁₂ + 0.325 𝐞₁₃ + 0.269 𝐞₁₄ + 0.146 𝐞₂₃ + 0.314 𝐞₂₄ + 0.523 𝐞₃₄ + 0.916 𝐞₁₂₃ + 0.815 𝐞₁₂₄ + 0.0614 𝐞₁₃₄ + 0.85 𝐞₂₃₄ + 0.201 𝐞₁₂₃₄,
 0.0862 + 3.92 𝐞₁ + 7.79 𝐞₂ + 0.422 𝐞₃ + -6.42 𝐞₄ + -4.26 𝐞₁₂ + 5.2 𝐞₁₃ + 9.13 𝐞₁₄ + 7.61 𝐞₂₃ + 5.29 𝐞₂₄ + 7.53 𝐞₃₄ + 1.31e+02 𝐞₁₂₃ + 1.99e+02 𝐞₁₂₄ + 77.6 𝐞₁₃₄ + -6.01 𝐞₂₃₄ + 60.1 𝐞₁₂₃₄)

In [111]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.0862 + 0.419 𝐞₁ + 0.81 𝐞₂ + 0.859 𝐞₃ + 0.0122 𝐞₄ + 0.182 𝐞₁₂ + 0.325 𝐞₁₃ + 0.269 𝐞₁₄ + 0.146 𝐞₂₃ + 0.314 𝐞₂₄ + 0.523 𝐞₃₄ + 0.916 𝐞₁₂₃ + 0.815 𝐞₁₂₄ + 0.0614 𝐞₁₃₄ + 0.85 𝐞₂₃₄ + 0.201 𝐞₁₂₃₄,
 0.0862 + 0.419 𝐞₁ + 0.81 𝐞₂ + 0.859 𝐞₃ + 0.0122 𝐞₄ + 0.182 𝐞₁₂ + 0.325 𝐞₁₃ + 0.269 𝐞₁₄ + 0.146 𝐞₂₃ + 0.314 𝐞₂₄ + 0.523 𝐞₃₄ + 0.916 𝐞₁₂₃ + 0.815 𝐞₁₂₄ + 0.0614 𝐞₁₃₄ + 0.85 𝐞₂₃₄ + 0.201 𝐞₁₂₃₄)

In [112]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(-0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄ + -8.94e-18 𝐞₁₂₃ + 4.57e-18 𝐞₁₂₄ + -1.56e-18 𝐞₁₃₄ + 1.36e-17 𝐞₂₃₄,
 -0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄ + -1.84e-17 𝐞₁₂₃ + 1.46e-18 𝐞₁₂₄ + -1.78e-17 𝐞₁₃₄ + -1.21e-16 𝐞₂₃₄)

In [113]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([-3.3e-17 + -0.0647 𝐞₁₂ + -0.983 𝐞₁₃ + 0.0616 𝐞₁₄ + 0.0565 𝐞₂₃ + -0.0132 𝐞₂₄ + -0.147 𝐞₃₄,
  -1.39e-17 + 0.0476 𝐞₁₂ + 0.149 𝐞₁₃ + 0.0361 𝐞₁₄ + 0.635 𝐞₂₃ + -0.0861 𝐞₂₄ + -0.751 𝐞₃₄,
  2.22e-16 + 0.147 𝐞₁₂ + -0.0132 𝐞₁₃ + -0.0565 𝐞₁₄ + -0.0616 𝐞₂₃ + -0.983 𝐞₂₄ + 0.0647 𝐞₃₄],
 [-0.628 𝐞₁₂ + -0.126 𝐞₁₃ + 0.753 𝐞₁₄ + 0.0449 𝐞₂₃ + -0.142 𝐞₂₄ + 0.0254 𝐞₃₄,
  -0.0476 𝐞₁₂ + -0.149 𝐞₁₃ + -0.0361 𝐞₁₄ + -0.635 𝐞₂₃ + 0.0861 𝐞₂₄ + 0.751 𝐞₃₄,
  -0.0476 𝐞₁₂ + -0.149 𝐞₁₃ + -0.0361 𝐞₁₄ + -0.635 𝐞₂₃ + 0.0861 𝐞₂₄ + 0.751 𝐞₃₄,
  0.384 𝐞₁₂ + -0.0168 𝐞₁₃ + -0.256 𝐞₁₄ + -0.384 𝐞₂₃ + -0.768 𝐞₂₄ + -0.223 𝐞₃₄])

In [114]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([-0.986 𝐞₁ + 0.0531 𝐞₂ + -0.0535 𝐞₃ + 0.151 𝐞₄,
  -0.0584 𝐞₁ + 0.0688 𝐞₂ + 0.994 𝐞₃ + -0.0536 𝐞₄,
  -0.15 𝐞₁ + -0.638 𝐞₂ + -0.00533 𝐞₃ + -0.755 𝐞₄,
  0.0512 𝐞₁ + -0.765 𝐞₂ + 0.0902 𝐞₃ + 0.636 𝐞₄],
 [[-0.986 𝐞₁ + 0.0531 𝐞₂ + -0.0535 𝐞₃ + 0.151 𝐞₄,
   -0.986 𝐞₁ + 0.0531 𝐞₂ + -0.0535 𝐞₃ + 0.151 𝐞₄,
   -0.986 𝐞₁ + 0.0531 𝐞₂ + -0.0535 𝐞₃ + 0.151 𝐞₄],
  [0.154 𝐞₁ + 0.631 𝐞₂ + -0.0751 𝐞₃ + 0.757 𝐞₄,
   0.154 𝐞₁ + 0.631 𝐞₂ + -0.0751 𝐞₃ + 0.757 𝐞₄,
   -0.0584 𝐞₁ + 0.0688 𝐞₂ + 0.994 𝐞₃ + -0.0536 𝐞₄],
  [-0.0462 𝐞₁ + 0.12 𝐞₂ + 0.992 𝐞₃ + 0.00764 𝐞₄,
   -0.0462 𝐞₁ + 0.12 𝐞₂ + 0.992 𝐞₃ + 0.00764 𝐞₄,
   -0.15 𝐞₁ + -0.638 𝐞₂ + -0.00533 𝐞₃ + -0.755 𝐞₄],
  [0.0512 𝐞₁ + -0.765 𝐞₂ + 0.0902 𝐞₃ + 0.636 𝐞₄,
   0.0512 𝐞₁ + -0.765 𝐞₂ + 0.0902 𝐞₃ + 0.636 𝐞₄,
   0.0512 𝐞₁ + -0.765 𝐞₂ + 0.0902 𝐞₃ + 0.636 𝐞₄]])

In [115]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(-0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄ + -8.94e-18 𝐞₁₂₃ + 4.57e-18 𝐞₁₂₄ + -1.56e-18 𝐞₁₃₄ + 1.36e-17 𝐞₂₃₄,
 -0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄)

In [116]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(-0.058 + 0.0792 𝐞₁₂ + 0.126 𝐞₁₃ + 0.288 𝐞₁₄ + 0.254 𝐞₂₃ + -0.117 𝐞₂₄ + -0.775 𝐞₃₄ + -0.46 𝐞₁₂₃₄,
 -0.058 + 0.0792 𝐞₁₂ + 0.126 𝐞₁₃ + 0.288 𝐞₁₄ + 0.254 𝐞₂₃ + -0.117 𝐞₂₄ + -0.775 𝐞₃₄ + -0.46 𝐞₁₂₃₄)

In [117]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.887 + 0.406 𝐞₁₂ + -0.0589 𝐞₁₃ + -0.128 𝐞₁₄ + -0.146 𝐞₂₃ + 0.0637 𝐞₂₄ + -0.0551 𝐞₃₄,
 0.887 + 0.406 𝐞₁₂ + -0.0589 𝐞₁₃ + -0.128 𝐞₁₄ + -0.146 𝐞₂₃ + 0.0637 𝐞₂₄ + -0.0551 𝐞₃₄)

In [118]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(-0.54 𝐞₁ + 0.411 𝐞₂ + 0.529 𝐞₃ + 0.734 𝐞₄,
 -0.54 𝐞₁ + 0.411 𝐞₂ + 0.529 𝐞₃ + 0.734 𝐞₄)

In [119]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(-0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄ + -8.94e-18 𝐞₁₂₃ + 4.57e-18 𝐞₁₂₄ + -1.56e-18 𝐞₁₃₄ + 1.36e-17 𝐞₂₃₄,
 -0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄,
 -0.501 𝐞₁ + 0.158 𝐞₂ + -0.791 𝐞₃ + 0.614 𝐞₄)

In [120]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(0.856 𝐞₁₂₃ + -0.496 𝐞₁₂₄ + 0.108 𝐞₁₃₄ + 0.0982 𝐞₂₃₄,
 0.856 𝐞₁₂₃ + -0.496 𝐞₁₂₄ + 0.108 𝐞₁₃₄ + 0.0982 𝐞₂₃₄)

In [121]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(0.0853 𝐞₁₂ + -0.218 𝐞₁₃ + -0.00151 𝐞₁₄ + 0.949 𝐞₂₃ + 0.484 𝐞₂₄ + -1.22 𝐞₃₄,
 0.0853 𝐞₁₂ + -0.218 𝐞₁₃ + -0.00151 𝐞₁₄ + 0.949 𝐞₂₃ + 0.484 𝐞₂₄ + -1.22 𝐞₃₄)

In [122]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(-1.04 𝐞₁ + 0.644 𝐞₂ + -0.388 𝐞₃ + 0.0242 𝐞₄ + -0.517 𝐞₁₂₃ + -0.822 𝐞₁₂₄ + 0.0601 𝐞₁₃₄ + 0.281 𝐞₂₃₄,
 -1.04 𝐞₁ + 0.644 𝐞₂ + -0.388 𝐞₃ + 0.0242 𝐞₄ + -0.517 𝐞₁₂₃ + -0.822 𝐞₁₂₄ + 0.0601 𝐞₁₃₄ + 0.281 𝐞₂₃₄)

In [123]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = create_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-2.98 + -6.94e-18 𝐞₁₂ + -1.39e-17 𝐞₁₃ + 2.78e-17 𝐞₂₄ + -1.39e-17 𝐞₂₅ + -1.66 𝐞₁₂₃₄ + 1.27 𝐞₁₂₃₅ + 0.924 𝐞₁₂₄₅ + -0.274 𝐞₁₃₄₅ + -1.31 𝐞₂₃₄₅

In [124]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.653 𝐞₁ + 0.91 𝐞₂ + 0.954 𝐞₃ + 0.494 𝐞₄ + 0.243 𝐞₅,
 0.653 𝐞₁ + 0.91 𝐞₂ + 0.954 𝐞₃ + 0.494 𝐞₄ + 0.243 𝐞₅)

In [125]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part

skew_blades = bivector_split(ortho_curl/2, alg)
sym_vec = reduce(alg.ip, skew_blades, alg.pseudoscalar((1,))).normalized()
ortho(sym_vec), sym_vec

(-0.495 𝐞₁ + 0.104 𝐞₂ + 0.348 𝐞₃ + -0.478 𝐞₄ + -0.628 𝐞₅,
 -0.495 𝐞₁ + 0.104 𝐞₂ + 0.348 𝐞₃ + -0.478 𝐞₄ + -0.628 𝐞₅)

In [126]:
rotors = []
for blade in skew_blades:
    t = P(sum(alg.frame), blade)
    blade_norm = norm(blade)
    # we can't simply rely on ortho_m, 
    # e.g. angle = np.arcsin(blade_norm) has an ambiguity for rotors.
    # Since (pi - angle) also generates the same sine/cosine square, but differnt rotors.
    # +-angle generates the same cosine, but blade.normalized() determines the orientation
    
    angle = np.arccos(np.median(terms_ratio(ortho(t) - (t | ortho_curl)/2, t)))
    half_blade_ortho = (angle/blade_norm/2) * blade.reverse()
    rotors.append(blade_exp(half_blade_ortho))
rotor = reduce(alg.gp, rotors)
rotor2 = sym_vec * rotor
xd = P(x, sym_vec.dual())
-rotor2.sw(x), ortho(x)

(-0.449 𝐞₁ + -0.563 𝐞₂ + -0.291 𝐞₃ + -1.23 𝐞₄ + 0.585 𝐞₅ + -4.86e-17 𝐞₁₂₃₄₅,
 -0.166 𝐞₁ + -0.622 𝐞₂ + -0.489 𝐞₃ + -0.961 𝐞₄ + 0.943 𝐞₅)

In [127]:
# On the other hand, we can solve the rotor part by Lasenby's method
frame = [item for blade in skew_blades for item in blade_split(blade, alg)]
r_frame = reciprocal(frame)
lasenby_rotor = sum(char_multi_linear(ortho, s, alg, frame, r_frame) for s in range(5))
terms_ratio(rotor.reverse(), lasenby_rotor)

array([0.29582615, 0.29582615, 0.29582615, 0.29582615, 0.29582615,
       0.29582615, 0.29582615, 0.29582615, 0.29582615, 0.29582615,
       0.29582615, 0.29582615, 0.29582615, 0.29582615, 0.29582615,
       0.29582615])

In [128]:
# For orthogonal transform, we simply normalize it
l_rotor = lasenby_rotor/norm(lasenby_rotor)
-(l_rotor * sym_vec).reverse().sw(x), ortho(x)

(-0.449 𝐞₁ + -0.563 𝐞₂ + -0.291 𝐞₃ + -1.23 𝐞₄ + 0.585 𝐞₅ + -2.08e-17 𝐞₁₂₃₄₅,
 -0.166 𝐞₁ + -0.622 𝐞₂ + -0.489 𝐞₃ + -0.961 𝐞₄ + 0.943 𝐞₅)